# Analyse and fix TFCheckpoint database

In [ ]:
# TODO - I did a more thorough analysis of TFCHeckpoint. Search for it and write it here
# TODO - Do not include this file in the finalised repo. Move elsewhere

In [ ]:

import pandas as pd
from IPython.display import display, HTML


# Custom functions
import sys
sys.path.append('../common')
from notebook_utils import table_of_contents, table_from_dict, h3, h4, h5, md
pd.set_option('display.max_colwidth', 20)

table_of_contents('fix_TFCheckpoint.ipynb')


<h3>Table of contents</h3>

[Analyse and fix TFCheckpoint database](#Analyse-and-fix-TFCheckpoint-database)

In [5]:
# PATHS
folder = '../../data/external/TF_id/'
TFCheckpoint_path = folder + "TFCheckpoint.tsv"

In [8]:
## LOAD & PREPROCESS TFCHECKPOINT TSV
# Load TFCheckpoint dataset
TF_df = pd.read_csv(TFCheckpoint_path, sep='\t', header=0)
str_cols = ['Associated.Gene.Name', 'Synonyms', 'Official name', 'Entrez.Taxa.ID', 'Entrez.Gene.ID', 'UniProt.SwissProt.Accession', 'Ensembl.Gene.ID']
TF_df[str_cols] = TF_df[str_cols].astype(str)


# Split Entrez, Taxa & UniProt into individual IDs
TF_df['EntrezIDs'] = TF_df['Entrez.Gene.ID'].str.split('|')
TF_df['TaxaIDs'] = TF_df['Entrez.Taxa.ID'].str.split('|')
TF_df['UniProt'] = TF_df['UniProt.SwissProt.Accession'].str.split('|')
TF_df['Ensembl'] = TF_df['Ensembl.Gene.ID'].str.split('|')

# Explode the TF
TF_exploded = TF_df.explode(['EntrezIDs', 'TaxaIDs', 'UniProt', 'Ensembl'])
TF_exploded = TF_exploded[TF_exploded["EntrezIDs"] != ''] # Drop empty rows (Appeared when | was present at the end, e.g. "9454|3425|")
TF_exploded = TF_exploded[TF_exploded["UniProt"] != '']
TF_exploded = TF_exploded[TF_exploded["Ensembl"] != '']

# Check whether each EntrezID only matches to 1 TaxaID:
gene_taxa_unique = TF_exploded.drop_duplicates(subset=["EntrezIDs", "TaxaIDs"], keep='first')
gene_taxa_mismatch = gene_taxa_unique[gene_taxa_unique.duplicated(subset=["EntrezIDs"], keep=False)]
h4("EntrezIDs mapped to 2 species")
md(f"There are {len(gene_taxa_mismatch['EntrezIDs'].unique())} Entrez IDs that are mapped to both Rat and Mouse:")
display(HTML(gene_taxa_mismatch[["EntrezIDs", "Associated.Gene.Name", "TaxaIDs"]].sort_values(by=['EntrezIDs']).to_html(index=False)))

# DROP MISMATCHING ROWS
rows_to_drop = [
    ['STAT5A', '20851', '10116'],
    ['STAT5A', '25126', '10090'],
    ['ZFY', '367832', '10090'],
    ['ZFY', '22764', '10116'],
    ['STAT5B', '24918', '10116']
]
for row in rows_to_drop:
    to_drop = (TF_exploded["Associated.Gene.Name"] == row[0]) & (TF_exploded["EntrezIDs"] == row[1])
    assert to_drop.sum() == 1, f"{to_drop.sum()} rows are being dropped instead of 1"
    TF_exploded = TF_exploded[~to_drop]
to_change = (TF_exploded["Associated.Gene.Name"] == "STAT5A") & (TF_exploded["EntrezIDs"] == "24918")
assert to_change.sum() == 1, f"{to_change.sum()} rows are being dropped instead of 1"
TF_exploded.loc[to_change, "TaxaIDs"] = "10116"
md("They have been searched in the NCBI and corrected manually")

# Assert there's no duplicates anymore
gene_taxa_unique = TF_exploded.drop_duplicates(subset=["EntrezIDs", "TaxaIDs"], keep='first')
gene_taxa_mismatch = gene_taxa_unique[gene_taxa_unique.duplicated(subset=["EntrezIDs"], keep=False)]
assert len(gene_taxa_mismatch) == 0, f"There's still {len(gene_taxa_mismatch)} duplicated rows" 

<h4>EntrezIDs mapped to 2 species</h4>

There are 5 Entrez IDs that are mapped to both Rat and Mouse:

EntrezIDs,Associated.Gene.Name,TaxaIDs
20851,STAT5A,10116
20851,STAT5B,10090
22764,ZFX,10090
22764,ZFY,10116
24918,STAT5A,10090
24918,STAT5B,10116
25126,STAT5A,10090
25126,STAT5B,10116
367832,ZFX,10116
367832,ZFY,10090


They have been searched in the NCBI and corrected manually

In [9]:
# GROUP DUPLICATED ROWS & GET FINAL TFCHECKPOINT DATASET
# In some rows, EntrezID, TaxaID & Name are the same -> Only SwissProt changes. We will group those rows

# Remove all useless columns
columns_to_keep = TF_exploded.columns.tolist()
columns_to_remove = ['Entrez.Taxa.ID', 'Entrez.Gene.ID', 'UniProt.SwissProt.Accession', 'Ensembl.Gene.ID', 'UniProt', 'Ensembl']
for column in columns_to_remove:
    columns_to_keep.remove(column)

# Group duplicated rows, with a | in between for UniProt & Ensembl.
TF_exploded = TF_exploded.groupby(columns_to_keep, dropna=False).agg({
    "UniProt": lambda x: "|".join(x),
    "Ensembl": lambda x: "|".join(x)
}).reset_index()

# Display one example
mask = TF_exploded["UniProt"].str.contains("\|")
md(f"In {mask.sum()} TFs, one EntrezID is mapped to 2 different SwissProt Accession IDs. They have been joined by |. Example:")
display(HTML(TF_exploded[mask][:2][["Associated.Gene.Name", "Official name", "EntrezIDs", "TaxaIDs", "UniProt", "Ensembl"]].to_html(index=False)))

In 16 TFs, one EntrezID is mapped to 2 different SwissProt Accession IDs. They have been joined by |. Example:

Associated.Gene.Name,Official name,EntrezIDs,TaxaIDs,UniProt,Ensembl
ABL1,Tyrosine-protein kinase ABL1,100909750,10116,E9PT20|F1M0A6,ENSRNOG00000009371|ENSRNOG00000009371
CHCHD2,Coiled-coil-helix-coiled-coil-helix domain-containing protein 2,316643,10116,Q5BJB3|M0R785,ENSRNOG00000051180|ENSRNOG00000051180
